# Deploy a pretrained sentiment model from Watson NLP as a Python function

This notebook demonstrates the steps needed to create and deploy a Python function that predicts sentiment in Watson Machine Learning. In this example, you also create an _online deployment_ of the Python function so you can invoke the sentiment analysis using a web service in an application.

## Table of Contents 

1. [Prerequisites](#prereq)
1. [Setup](#setup)
1. [Create a Python function](#function)
1. [Save the Python function to the Model repository](#modelrepo)
1. [Create an Online Deployment](#deployment)
1. [Predict the sentiment of sample data](#predict)
1. [Clean up](#cleanup)
1. [Summary](#summary)

<a id="prereq"></a>
## Prerequisites 

Before running this notebook, you need to create a new deployment space or identify an existing deployment space to use for this example.

<a id="setup"></a>
## 1. Set up the environment
Before you use the sample code in this notebook, you must perform the following setup tasks:

- Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).
- Install and import ibm-watsonx-ai and its dependencies by using the code cell below.

Note: See the ibm-watsonx-ai documentation [here](https://ibm.github.io/watsonx-ai-python-sdk/index.html).

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1

### Connect to WML
To authenticate the Watson Machine Learning service on IBM Cloud, you need to provide the platform api key and instance location.

You can use the IBM Cloud CLI to retrieve your platform API Key and instance location.

Generate your API Key:

In [ ]:
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME

As a result, you get the api_key value from the output.

Retrieve the location of your WML instance:

In [ ]:
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME

As a result, you get the value of the location from the output.

Tips: 

- You can also generate your Cloud API key by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the API Keys section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it in the code cell below. 
- You can get a service specific URL by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning). Check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.
- You can get a service specific API key by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids). From that page, click **Create**, then copy the created key and paste it in the code cell below.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    api_key=api_key,
    url='https://' + location + '.ml.cloud.ibm.com'
)

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [ ]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Set up a space
First, create a space that will be used for your work. If you do not have space already created, you can use Deployment Spaces Dashboard to create one.

1. Click **New Deployment Space**
1. Create an empty space
1. Select **Cloud Object Storage**
1. Select **Watson Machine Learning** instance and click **Create**
1. Copy the `space_id` and paste it in the code below

Tip: You can also use SDK to prepare the space for your work. More information can be found [here](https://ibm.github.io/watsonx-ai-python-sdk/core_api.html#client.PlatformSpaces).

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use the list method to print all of the existing spaces.

In [ ]:
client.spaces.list(limit=10)

Set a default space to interact with all the resources available in Watson Machine Learning.

In [ ]:
client.set.default_space(space_id)

<a id="function"></a>
## 2. Create a Python function that detects sentiment

The Python function for this notebook uses the targeted sentiment block `targets-sentiment_transformer-workflow_multilingual_slate.153m.distilled-cpu` that is provided by Watson NLP.

In [ ]:
def detect_sentiment():
    import watson_nlp

    sentiment_model = targets_sentiment_model = watson_nlp.load('targets-sentiment_transformer-workflow_multilingual_slate.153m.distilled-cpu')

    def score(input):
        scoring_prediction_out = []
        for input_data_row in input["input_data"][0]["values"]:
            scoring_prediction_row = []
            for input_data in input_data_row:
                targets_sentiments = targets_sentiment_model.run(input_data)
                scoring_prediction_row.append(targets_sentiments.to_dict())

            scoring_prediction_out.append(scoring_prediction_row)

        # Score using the pre-defined model
        scoring_response = {
            'predictions': [{'fields': ['nlp_prediction'],
                             'values': scoring_prediction_out
                            }]
        }

        return scoring_response

    return score

<a id="modelrepo"></a>
## 3. Save the Python function to the Model repository

In [ ]:
pyfunc_swspec_id = client.software_specifications.get_uid_by_name("runtime-24.1-py3.11")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'sentiment_function',
    client.repository.FunctionMetaNames.DESCRIPTION: 'sentiment_function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}


sentiment_function_details = client.repository.store_function(meta_props=meta_data, function=detect_sentiment)

<a id="deployment"></a>
## 4. Create an Online Deployment

In [ ]:
sentiment_function_uid = client.repository.get_function_id(sentiment_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "nlp-sentiment-deploy",
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "nlp-sentiment-deploy",
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},
   client.deployments.ConfigurationMetaNames.ONLINE: {   }
}

sentiment_deployment_details = client.deployments.create(sentiment_function_uid, meta_props=meta_props)
sentiment_deployment_id = client.deployments.get_id(sentiment_deployment_details)

.

.

.

.

.

...

.

.

.

.


ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4b6b02b2-d256-45ae-b683-c5d458fca6c4'
------------------------------------------------------------------------------------------------


{'entity': {'asset': {'id': '49dbf26f-eb7f-47ad-84d3-cb4101d4e359'}, 'custom': {}, 'deployed_asset_type': 'function', 'description': 'nlp-sentiment-deploy', 'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b', 'name': 'S', 'num_nodes': 1}, 'name': 'nlp-sentiment-deploy', 'online': {}, 'space_id': '4a2a89fb-9bd4-4e27-9301-372a71d1872f', 'status': {'inference': [{'url': 'https://cpd-dev.apps.midgard.cp.fyre.ibm.com/ml/v4/deployments/4b6b02b2-d256-45ae-b683-c5d458fca6c4/predictions'}], 'online_url': {'url': 'https://cpd-dev.apps.midgard.cp.fyre.ibm.com/ml/v4/deployments/4b6b02b2-d256-45ae-b683-c5d458fca6c4/predictions'}, 'serving_urls': ['https://cpd-dev.apps.midgard.cp.fyre.ibm.co

<a id="predict"></a>
## 5. Predict the sentiment of sample data

In [ ]:
scoring_payload = {
   client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': ["Text"],
        'values': [["The room is nice, but the price is too expensive. The window frame is made of wood."],
                   ["The check-in was horrible."],
                  ]
   }]
}

In [ ]:
predictions = client.deployments.score(sentiment_deployment_id, scoring_payload)
predictions

<a id="cleanup"></a>
## 6. Clean up

To clean up the created assets, run the code below.

In [ ]:
client.deployments.delete(sentiment_deployment_id)
client.repository.delete(sentiment_function_uid)

For more information on artifact cleanup, see [Machine Learning artifacts management sample notebook](https://dataplatform.cloud.ibm.com/exchange/public/entry/view/55ef73c276cd1bf2bae266613d08c0f3?context=cpdaas).

<a id="summary"></a>
## 7. Summary and next steps

You successfully completed this notebook! You learned how to use Watson Machine Learning for function deployment and scoring with NLP.

Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx) for more samples, tutorials, documentation, how-tos, and blog posts.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.